In [1]:
import os
import sys
os.environ["SPARK_HOME"] = "/usr/lib/spark/"
os.environ["PYSPARK_PYTHON"]= "/home/cloudera/anaconda2/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"]= "/home/cloudera/anaconda2/bin/python"
os.environ["SPARK_YARN_USER_ENV"]= "/home/cloudera/anaconda2/bin/python"

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.1-src.zip'))
execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

from pyspark import SparkContext, SparkConf

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.6.0
      /_/

Using Python version 2.7.13 (default, Dec 20 2016 23:09:15)
SparkContext available as sc, HiveContext available as sqlContext.


In [2]:
joined_prior =  sc.textFile("/user/cloudera/kaggle/order_product_prior_joined")

In [4]:
def tuplesort(a,b,c,d,e):
    combined = list()
    x,y,z,s,t=(),(),(),(),()
    for i in range(len(b)):
        combined.append((a[i],int(b[i]),c[i],d[i],e[i]))    
    j= sorted(combined, key= lambda x : x[1])
    for i in range(len(combined)):
        x,y,z,s,t= x + (j[i][0],), y+(j[i][1],), z+ (j[i][2],), s + (j[i][3],), t +(j[i][4],)
    return [x,y,z,s,t]

In [5]:
inter_join=joined_prior.map(lambda x : x.split(',')).map(lambda x : (int(x[1]), (x[0], x[3],x[4],x[6],x[7] ))).\
    combineByKey((lambda x : ((x[0],), (x[1],),(x[2],),(x[3],), (x[4],))), 
                 (lambda x,y: (x[0] + (y[0],),x[1] + (y[1],),x[2] + (y[2],),x[3] + (y[3],),x[4] + (y[4],))),
                 (lambda x,y : (x[0]+y[0],x[1]+y[1],x[2]+y[2],x[3]+y[3],x[4]+y[4])))

In [11]:
inter1_join=inter_join.map(lambda x : (x[0], tuplesort(x[1][0], x[1][1], x[1][2],x[1][3],x[1][4])))\
    .map(lambda x : (x[0],' '.join(x[1][0]), ' '.join(x[1][2]),' '.join(x[1][3]),'|'.join(x[1][4]))).\
    map(lambda x : (x[0], x[4]))

In [14]:
inter1_join.map(lambda x : (str(x[0]),x[1])).map(lambda x: ','.join(x)).saveAsTextFile("/user/cloudera/kaggle/user_prod_sort")

In [15]:
product = sc.textFile("file:///home/cloudera/Desktop/kaggle/products.csv",3)

In [41]:
prod_sorted = product.filter(lambda x : 'product_id' not in x).map(lambda x : x.split(',')).map(lambda x : int(x[0])).sortBy(lambda x : x)

In [45]:
user_prod_sort=sc.textFile("/user/cloudera/kaggle/user_prod_sort")

In [48]:
user_prod_sort.map(lambda x : x.split(',')).map(lambda x: x[1]).take(1)

[u'1511 18523 9934 47209 44473 4920 13740 40910 21290 21137 44275 49235 28204 4793 33497 10895 41950 25072 48230 28993 5194 38313 42383 15700 5077 39921 2450 35108 31263 43789 21903 41544 18610 49174 21461 30336 39928 37220 17979|1511 21137 5194 2770 44275 4920 13740 47209 6631 38313 25072 4793 28204 21290 895 38453 32293 33135 39928 24841 24561 17484|24964 22935 43076 19519 45535 38274|1511 21137 5194 4920 44275 33497 9934 38313 28204 25072 4793 44473 895 40910 2452 39921 22170 20327 18610 7806 2081 4659 47766 17484 39781|49235 10749 44359 39275 21137 26209 28842 33746 3376|1511 21137 5194 49235 44275 44473 6631 2452 38313 40910 895 39921 22170 20327 10132 24841 39928 9076 21543 12204 3481 47766 17484 36772 18610 1896|2770 47209 34023 28849 3376 1999 5818 19678 31343 16570 39928 28204 12254 38274|2770 1511 21137 5194 44275 47209 44473 40910 28204 2452 38313 895 25072 22170 39921 33135 3481 18610 18523 35108 36036 16093 48720 21019 1896 26344|34551 3481 35108 24841 1896 26344 38641 214

In [50]:
prod_seq= '1511 18523 9934 47209 44473 4920 13740 40910 21290 21137 44275 49235 28204 4793 33497 10895 41950 25072 48230 28993 5194 38313 42383 15700 5077 39921 2450 35108 31263 43789 21903 41544 18610 49174 21461 30336 39928 37220 17979|1511 21137 5194 2770 44275 4920 13740 47209 6631 38313 25072 4793 28204 21290 895 38453 32293 33135 39928 24841 24561 17484|24964 22935 43076 19519 45535 38274|1511 21137 5194 4920 44275 33497 9934 38313 28204 25072 4793 44473 895 40910 2452 39921 22170 20327 18610 7806 2081 4659 47766 17484 39781|49235 10749 44359 39275 21137 26209 28842 33746 3376|1511 21137 5194 49235 44275 44473 6631 2452 38313 40910 895 39921 22170 20327 10132 24841 39928 9076 21543 12204 3481 47766 17484 36772 18610 1896|2770 47209 34023 28849 3376 1999 5818 19678 31343 16570 39928 28204 12254 38274|2770 1511 21137 5194 44275 47209 44473 40910 28204 2452 38313 895 25072 22170 39921 33135 3481 18610 18523 35108 36036 16093 48720 21019 1896 26344|34551 3481 35108 24841 1896 26344 38641 21461 35045 49174 9808 1700 12980 6873 39430 14267|4920 4793 21137 44275 38313 1896 38641 26344|34448 22888 10761 9595 48230 32293 22935 45535 4920 44359 10132 1511 21137 22170|13740|37029 48230 2450 49235 5194|9598 4138 10761 42383 13740 24561 22935 4920 44359 10132 1511 44275 2452 895 28204 3481 1896 26344|9934 49235 39928 1511 28204|28985 32175 12916 45066 26790 12144 13740 4793 42265 39928 1511 44275 18610 35108|26790 13740 38453 4920 44359 47766 42265 1511 44275 44473 2452 1896 26344|4920 49235 1511|40604 34126 22474 4138 33198 44375 41220 27104 24964 31343 1511 25072 18610 35108 21461 49174|37119 22888 44375 10895 48230 28993 2450 32293 1511 22170 39921 18610 35108 12980|44912|41220 27104 24964 1511 21461 49174|22474 15143 44375 10761 13740 49235 25072 18610|18200 38383 39561 12144 37220 13740 44359 47766 17484 42265 1511 2770 5194 25072 22170 24841|45066 24852 2770 20995 31343 1999 18479 38562|1511 47766 13176 49235 2770 44275 28204 19048 581 34448 3599 26790 16145 8670 47626 28985|1511 21137 24852 4920 2452 2770 3599 22474 34448 31869 49235 25072 6552 8056 28204|37220 13176 28204 42265 7751 48416 34942 21603 21108 3376 44359 39558 27104 39984 22935 39921 34635 42701|24891 1511 13176 47766 48679 44375 21108 22825 4920 39928|2770 13176 28204 4920 47766 26790 2452 44275 44375 25072 5194 39921 18610 42265 22170 21137 10761 26344 1896 10132 45840 44116 4138 9092 33198 4957 31869 43442 3481 23543 6631 21019 31040 41787 36772 27104 46720 27663 3376 32906 5876 2447|49235 47766 13176 21137 40706 20536 44035 43789 4056 10948|1511 11736 8988 30391 44359 19677 19057 21137 44422 21938 2295 13212|49235 37067 8277 47209 5077 33746 8006 45007 23400 5818 19677 22504 23543 6647 39928 31263 13740 39561 25072 18523 37220|9934 5077 1511 37067 47766 45066 2452 39928 24841 26790 42265 31040 27104 1999|18696 28849 14267 13176 47766 2770 1511 49235 40516 40706 30391 3989|13176 19677 28204 47209 48679 15040 784 10163 44422|9934 1511 24891 49235 31040 13176 32175 21603|47766 28204 2770 13740|1511 13176 47766 2770 49235 13740 24891 2452 9934 42265 4920 44275 10132 18523 23543 4957 11736 22888 6873 10895 12980 2450 4163 48325 42383 2651|21290 13380 1511 13176 34448'

In [58]:
prod = prod_sorted.collect()

In [56]:
sc.broadcast(prod)

In [68]:
NotebookApp.iopub_data_rate_limit=10000000000

NameError: name 'NotebookApp' is not defined

In [75]:
t = (0,0)
list1= []
for _ in range(len(prod)):
    list1.append(t)
from collections import defaultdict
user_trans = defaultdict()
for x in range(1,10):
    user_trans[x]= list1
user_trans

TypeError: first argument must be callable or None

In [53]:
# def trans_matrix(prod_seq):
basket_seq = prod_seq.split('|')
iteration = len(basket_seq)
for j in range(iteration-1):
    trans_from = basket_seq[j]
    trans_to = basket_seq[j+1]
    for i in range(len(prod)):

40